## Imports

In [ ]:
from scripts.download import Download_LUNA25 as luna25

In [ ]:
downloader = luna25()

## Downloads the Record Images
- Warning: ~212GB (For All Files...)

In [ ]:
downloader.download()

## Unzip the Files

In [ ]:
downloader.extract("data", "extracted")